In [14]:
# First, add your HuggingFace Token (with Write access) to Google Colab Secrets
# Then, load the token using below code

from huggingface_hub import login
from google.colab import userdata

hf_token = userdata.get('HF_API_KEY')
login(token = hf_token)

In [15]:
#@title Download Dataset
from IPython.display import clear_output
!gdown https://drive.google.com/uc?id=1Q1r1FGDaBQLkvPKMjvgnA2eWBpmMLD57
clear_output()
!ls | grep '.csv'

clean_MedQuAD_autotrain_ready.csv


In [16]:
import pandas as pd

df = pd.read_csv("clean_MedQuAD_autotrain_ready.csv")
df.shape
df.head()

,Question,Answer
0,what is (are) age-related macular degeneration ?,"age-related macular degeneration, also known a..."
1,what are the treatments for age-related macula...,"wet amd can be treated with laser surgery, pho..."
2,what is (are) age-related macular degeneration ?,the daily amounts used by the study researcher...
3,who is at risk for age-related macular degener...,risk increases with age amd is most common in ...
4,what is (are) age-related macular degeneration ?,drusen are yellow deposits under the retina. t...


In [17]:
# Load your fine-tuned model

from sentence_transformers import SentenceTransformer

# Load embedding model
fine_tuned_model = SentenceTransformer("rajucanon/mini-project-medical-assistant2")      # <------ change this as per your model id
                                                                                # This finetuned model is standing on
                                                                                # Base Model: sentence-transformers/all-MiniLM-L6-v2


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/199 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [18]:
# Load your fine-tuned model

from sentence_transformers import SentenceTransformer

# Load embedding model
untuned_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [19]:
def compute_embeddings(x, model):
    return model.encode(x)


In [20]:
# Create Fine-tuned embeddings & Untuned embeddings for the `Question`

df["fine_tuned_embeddings"] = df["Question"].apply(lambda x: compute_embeddings(x, fine_tuned_model))
df["untuned_embeddings"] = df["Question"].apply(lambda x: compute_embeddings(x,untuned_model))

print("Embeddings ready for", len(df), "questions.")

Embeddings ready for 520 questions.


In [10]:
df.head()

,Question,Answer
0,what is (are) age-related macular degeneration ?,"age-related macular degeneration, also known a..."
1,what are the treatments for age-related macula...,"wet amd can be treated with laser surgery, pho..."
2,what is (are) age-related macular degeneration ?,the daily amounts used by the study researcher...
3,who is at risk for age-related macular degener...,risk increases with age amd is most common in ...
4,what is (are) age-related macular degeneration ?,drusen are yellow deposits under the retina. t...


In [21]:
import numpy as np

def get_answer(user_question, model, top_k=1):
    # Encode user question
    q_emb = model.encode(user_question)

    if model == fine_tuned_model:
        # Convert embeddings column to numpy array
        all_embs = np.vstack(df["fine_tuned_embeddings"].values)

    else:
        # Convert embeddings column to numpy array
        all_embs = np.vstack(df["untuned_embeddings"].values)


    # Compute cosine similarities (vectorized for speed)
    sims = np.dot(all_embs, q_emb) / (
        np.linalg.norm(all_embs, axis=1) * np.linalg.norm(q_emb)
    )
    #----------------------------------
    # Get indices of top-k matches
    top_idx = np.argsort(sims)[::-1][:top_k]

    # Build response
    results = []
    for idx in top_idx:
        results.append(
            f"**Similarity: {sims[idx]:.2f}** → Answer: {df.iloc[idx]['Answer']}"
        )

# Return matched top 3 answers
    return "\n\n".join(results)


In [11]:
# Test the fine-tuned model

get_answer("Can lifestyle changes reduce the risk of developing type 2 diabetes?", model = fine_tuned_model)

NameError: name 'fine_tuned_model' is not defined

In [12]:
# Test the untuned model

get_answer("Can lifestyle changes reduce the risk of developing type 2 diabetes?", model = untuned_model)

NameError: name 'fine_tuned_model' is not defined

In [22]:
import gradio as gr

# Function to get output from Fine-tuned model
def get_response(input):
    response = get_answer(input, model = fine_tuned_model)
    return response


# Input element
in_ques = gr.Textbox(label="Ask a medical question", placeholder="Type your question here...", lines=5)

# Output element
out_ans = gr.Textbox(label="Answer", lines = 10)


# Create Gradio Interface
iface = gr.Interface(fn = get_response,
                     inputs = [in_ques],
                     outputs = [out_ans],
                     title = "Medical Q&A",
                     description = "Application to answer medical questions using a fine-tuned model",
                     flagging_mode = "never")
# Launch interface
iface.launch()          # set `debug=True` while debugging

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bdfb03ba6b8fa07bea.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
